## Dependencies

In [1]:
import torch
import torch.nn as nn

In [2]:
import easydict
from random import randint
from itertools import permutations

## Hyperparams

In [3]:
EMBD_DIM = 7
OUT_DIM = 4
BATCH = 10
N = 3

# Concept Vocabulary

In [4]:
def to_glove(token):
    return torch.rand(EMBD_DIM)

### Dummy data

In [5]:
property_types = ['color', 'material']

property_concepts = {
    'color': ['red', 'green', 'blue'],
    'material': ['cloth', 'rubber']
}

state_identities = ['cat', 'shirt']

relationships = ['holding', 'behind']

## Preparation

In [6]:
# each property has a type
L = len(property_types)

# we add identity and relations in idx 0 and L+1 respectively (TODO: with those names?)
property_types = ['identity'] + property_types
property_types += ['relations']
property_concepts['identity'] = state_identities
property_concepts['relations'] = relationships

D = torch.stack([to_glove(property_type) for property_type in property_types])

In [7]:
# each property has a series of concepts asociated
# ordered_C is separated by property, C includes all concepts.
ordered_C = [
    torch.stack([to_glove(concept) for concept in property_concepts[property]])
    for property in property_types
]
C = torch.cat(ordered_C, dim=0)

# we add c' for non structural words (@ idx -1)
# TODO: c' initialization?
c_prime = torch.rand(1, EMBD_DIM, requires_grad=True)
C = torch.cat([C, c_prime], dim=0)

# Scene Graph

### Dummy data

In [8]:
nodes = ['kitten', 'person', 'shirt']

relations = {
    ('person', 'shirt'): 'wear',
    ('person', 'kitten'): 'holding',
    ('kitten', 'shirt'): 'bite'
}

## Preparation

In [9]:
# for simplicity: random state initialization of properties (TODO)
S = torch.rand(BATCH, len(nodes), L+1, EMBD_DIM)

In [10]:
# build adjacency matrix (TODO: now all graphs are same)
# the edge features e' are inserted into an adjacency matrix for eficiency
adjacency_mask = torch.zeros(BATCH, len(nodes), len(nodes))
E = torch.zeros(BATCH, len(nodes), len(nodes), EMBD_DIM)
for idx_pair in permutations(range(len(nodes)), 2):
    pair = tuple(nodes[idx] for idx in idx_pair)
    if pair in relations:
        E[:,idx_pair[0],idx_pair[1]] = torch.rand(EMBD_DIM)   # (TODO)
        adjacency_mask[:,idx_pair[0],idx_pair[1]] = 1

In [90]:
# alternatively we can use hybrid tensors to reduce memory and computation overhead
indices = []
values = []
for idx_pair in permutations(range(len(nodes)), 2):
    pair = tuple(nodes[idx] for idx in idx_pair)
    if pair in relations:
        indices.append(idx_pair)
        values.append(torch.rand(EMBD_DIM))

sparse_adj = torch.sparse.FloatTensor(
    torch.LongTensor(indices).t(), 
    torch.stack(values),
    (len(nodes), len(nodes), EMBD_DIM)
)

In [82]:
sparse_adj.shape

torch.Size([3, 3, 7])

In [83]:
sparse_adj.coalesce().indices().shape

torch.Size([2, 3])

In [84]:
sparse_adj.coalesce().values().shape

torch.Size([3, 7])

In [85]:
3*3*7 > (2*3) + (3*7)

True

In [91]:
sparse_adj.resizeAs_()

AttributeError: 'Tensor' object has no attribute 'resizeAs_'